In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [9]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load dataset
train_data = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_train.csv", encoding='latin1')
test_data = pd.read_csv("/kaggle/input/email-classification-nlp/SMS_test.csv", encoding='latin1')

In [5]:
# Preprocessing
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stopwords_list]
    # Join tokens back into string
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

In [6]:
# Apply preprocessing to train and test data
print(train_data.columns)
train_data['Message_body'] = train_data['Message_body'].apply(preprocess_text)
test_data['Message_body'] = test_data['Message_body'].apply(preprocess_text)

Index(['S. No.', 'Message_body', 'Label'], dtype='object')


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['Message_body'])  # Replace 'text' with 'Message_body'
y_train = train_data['Label']  # Use 'Label' instead of 'label'
X_test = vectorizer.transform(test_data['Message_body'])  # Replace 'text' with 'Message_body'
y_test = test_data['Label']  # Use 'Label' instead of 'label'


In [10]:
# Train SVM classifier
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [11]:
# Predictions
y_pred = classifier.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [13]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.88

Classification Report:
              precision    recall  f1-score   support

    Non-Spam       0.77      1.00      0.87        49
        Spam       1.00      0.80      0.89        76

    accuracy                           0.88       125
   macro avg       0.88      0.90      0.88       125
weighted avg       0.91      0.88      0.88       125


Confusion Matrix:
[[49  0]
 [15 61]]
